In [1]:
# Installiere Ultralytics (falls noch nicht vorhanden)
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 23.8 MB/s eta 0:00:00


In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="OC1w5MMsOvtMMyW92Jp7")
project = rf.workspace("taleb-alhassan-kiofa").project("hrw_relevant")
version = project.version(8)
dataset = version.download("yolov8")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to HRW_Relevant-8 in yolov8:: 100%|██████████| 2212/2212 [00:00<00:00, 5462.80it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
import os
import random

# Funktion zum Mischen der Dateien in einem Ordner
def shuffle_files(image_dir, label_dir):
    image_files = sorted([f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])
    label_files = sorted([f for f in os.listdir(label_dir) if f.endswith('.txt')])

    # Überprüfen, ob die Anzahl der Bilder und Labels gleich ist
    assert len(image_files) == len(label_files), "Die Anzahl der Bilder und Labels stimmt nicht überein!"

    # Zufällige Reihenfolge erstellen
    indices = list(range(len(image_files)))
    random.shuffle(indices)

    # Dateien umbenennen
    for i, idx in enumerate(indices):
        new_image_name = f"{i:05d}.jpg"  # Benenne die Bilder um
        new_label_name = f"{i:05d}.txt"  # Benenne die Labels um

        os.rename(os.path.join(image_dir, image_files[idx]), os.path.join(image_dir, new_image_name))
        os.rename(os.path.join(label_dir, label_files[idx]), os.path.join(label_dir, new_label_name))

# Ordnerpfade definieren
train_image_dir = "/content/HRW_Relevant-8/train/images"
train_label_dir = "/content/HRW_Relevant-8/train/labels"

valid_image_dir = "/content/HRW_Relevant-8/valid/images"
valid_label_dir = "/content/HRW_Relevant-8/valid/labels"

test_image_dir = "/content/HRW_Relevant-8/test/images"
test_label_dir = "/content/HRW_Relevant-8/test/labels"

# Shuffle für Train, Valid, Test
shuffle_files(train_image_dir, train_label_dir)
shuffle_files(valid_image_dir, valid_label_dir)
shuffle_files(test_image_dir, test_label_dir)

print("Dateien erfolgreich gemischt!")

Dateien erfolgreich gemischt!


In [7]:
from ultralytics import YOLO
import os

# YOLOv8 Modell laden (YOLOv8n für schnelles Training)
model = YOLO('yolov8n.pt')

# Trainieren des Modells mit Augmentierungen
model.train(
    data='/content/HRW_Relevant-8/data.yaml',  # Pfad zur data.yaml-Datei
    epochs=50,                         # Anzahl der Epochen
    batch=16,                          # Batch-Größe
    imgsz=640,                         # Bildgröße
    augment=False,                      # Unaktiviert Augmentierungen
    project='/content/custom_training_results/train0',  # Projektordner
    name='train01',           # Name des spezifischen Trainings
    val=True                            # Validierung nach jeder Epoche
)

Ultralytics 8.3.44 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/HRW_Relevant-8/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/custom_training_results/train0, name=train01, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, 

100%|██████████| 22.2M/22.2M [00:00<00:00, 102MB/s] 

Overriding model.yaml nc=80 with nc=9

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

Model summary: 225 layers, 3,012,603 parameters, 3,012,587 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
TensorBoard: Start with 'tensorboard --logdir /content/custom_training_results/train0/train01', view at http://localhost:6006/
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅


train: Scanning /content/HRW_Relevant-8/train/labels.cache... 770 images, 26 backgrounds, 0 corrupt: 100%|██████████| 770/770 [00:00<?, ?it/s]

...

                   all        220        433      0.762      0.771      0.799      0.609



50 epochs completed in 0.259 hours.
Optimizer stripped from /content/custom_training_results/train0/train01/weights/last.pt, 6.3MB
Optimizer stripped from /content/custom_training_results/train0/train01/weights/best.pt, 6.3MB

Validating /content/custom_training_results/train0/train01/weights/best.pt...
Ultralytics 8.3.44 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,007,403 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:03<00:00,  2.14it/s]


                   all        220        433       0.73      0.784      0.801       0.61
        Links-abbiegen         42         45      0.685      0.822      0.812        0.7
       Rechts-abbiegen         34         36      0.571       0.75      0.716      0.556
            Ampel-grün         47         93      0.876      0.839      0.864      0.549
             Ampel-rot         29         49       0.63      0.556      0.625      0.377
            Ampel-gelb         23         46      0.911      0.887      0.922      0.608
             Geradeaus         44         49      0.876      0.867      0.878      0.716
  Geradeaus-oder-links         24         27      0.584      0.667      0.719      0.575
 Geradeaus-oder-rechts         37         40      0.666      0.796      0.742      0.613
         Zebrastreifen         30         48      0.773      0.875      0.935      0.793
Speed: 0.3ms preprocess, 2.3ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to /content/cu

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a4a5c7d0610>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.0470

In [8]:
model = YOLO('/content/custom_training_results/train0/train01/weights/best.pt')  # Pfad zum trainierten Modell

# Validierung durchführen
model.val(
    data='/content/HRW_Relevant-8/data.yaml',  # Pfad zur data.yaml-Datei
    save_json=True,                     # Ergebnisse im COCO-Format speichern
    project='/content/custom_training_results/train0/train01',  # Hauptordner für Ergebnisse
    name='Valid_01'           # Name des Validierungsordners
)

Ultralytics 8.3.44 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,007,403 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/HRW_Relevant-8/valid/labels.cache... 220 images, 10 backgrounds, 0 corrupt: 100%|██████████| 220/220 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:03<00:00,  3.97it/s]


                   all        220        433      0.733      0.787      0.803       0.61
        Links-abbiegen         42         45      0.685      0.822      0.811      0.701
       Rechts-abbiegen         34         36      0.571       0.75      0.716      0.566
            Ampel-grün         47         93      0.876      0.839      0.871      0.549
             Ampel-rot         29         49      0.653      0.576      0.641      0.374
            Ampel-gelb         23         46      0.911      0.887       0.92      0.601
             Geradeaus         44         49      0.877       0.87      0.878      0.719
  Geradeaus-oder-links         24         27      0.585      0.667      0.718      0.577
 Geradeaus-oder-rechts         37         40      0.667        0.8      0.742      0.623
         Zebrastreifen         30         48      0.772      0.875      0.935      0.781
Speed: 0.3ms preprocess, 4.7ms inference, 0.0ms loss, 2.5ms postprocess per image
Saving /content/custom_train

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a4a54d5f4c0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.0470

In [9]:

model = YOLO('/content/custom_training_results/train0/train01/weights/best.pt')  # Pfad zum trainierten Modell
results = model.predict(
    source='/content/HRW_Relevant-8/test/images',  # Pfad zu Testbildern
    conf=0.5,                              # Mindestkonfidenz
    save=True,                              # Ergebnisse speichern
    project='/content/custom_training_results/train0/train01',  # Hauptordner
    name='test_01',                    # Name des Testordners
)


image 1/110 /content/HRW_Relevant-8/test/images/00000.jpg: 640x640 2 Ampel-gelbs, 8.5ms
image 2/110 /content/HRW_Relevant-8/test/images/00001.jpg: 640x640 1 Zebrastreifen, 7.4ms
image 3/110 /content/HRW_Relevant-8/test/images/00002.jpg: 640x640 3 Ampel-rots, 1 Geradeaus, 7.9ms
image 4/110 /content/HRW_Relevant-8/test/images/00003.jpg: 640x640 1 Geradeaus-oder-links, 7.6ms
image 5/110 /content/HRW_Relevant-8/test/images/00004.jpg: 640x640 1 Rechts-abbiegen, 7.4ms
image 6/110 /content/HRW_Relevant-8/test/images/00005.jpg: 640x640 1 Links-abbiegen, 7.5ms
image 7/110 /content/HRW_Relevant-8/test/images/00006.jpg: 640x640 1 Ampel-gelb, 7.4ms
image 8/110 /content/HRW_Relevant-8/test/images/00007.jpg: 640x640 1 Geradeaus-oder-rechts, 7.4ms
image 9/110 /content/HRW_Relevant-8/test/images/00008.jpg: 640x640 1 Geradeaus-oder-rechts, 7.4ms
image 10/110 /content/HRW_Relevant-8/test/images/00009.jpg: 640x640 1 Ampel-grün, 1 Geradeaus, 1 Geradeaus-oder-links, 7.4ms
image 11/110 /content/HRW_Relevan

In [11]:

from ultralytics import YOLO
import os

# YOLOv8 Modell laden (YOLOv8n für schnelles Training)
model = YOLO('yolov8n.pt')


model.train(
    data='/content/HRW_Relevant-8/data.yaml',  # Pfad zur data.yaml-Datei
    epochs=50,                                 # Anzahl der Epochen
    batch=16,                                  # Batch-Größe
    imgsz=640,                                 # Bildgröße
    augment=True,                              # Aktiviert Augmentierungen
    degrees=5.0,                              # Rotation (±10 Grad)
    translate=0.2,                             # Translation (±20%)
    scale=0.5,                                 # Skalierung (±50%)
    hsv_h=0.015,                               # Farbtonanpassung (±1.5%)
    hsv_s=0.7,                                 # Sättigungsanpassung (±70%)
    hsv_v=0.4,                                 # Helligkeitsanpassung (±40%)
    mosaic=1.0,                                # Mosaic-Augmentierung aktiviert
    mixup=0.2,                                 # MixUp-Augmentierung aktiviert (20%)
    copy_paste=0.1,                            # CopyPaste-Augmentierung aktiviert (10%)
    fliplr=0,                                  # Deaktiviert horizontales Spiegeln
    flipud=0,
    shear =0,
    perspective=0,                            # Keine Perspektivänderung
    project='/content/custom_training_results/train0',  # Projektordner
    name='train02',           # Name des spezifischen Trainings
    val=True
)


Ultralytics 8.3.44 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/HRW_Relevant-8/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/custom_training_results/train0, name=train02, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, s

train: Scanning /content/HRW_Relevant-8/train/labels.cache... 770 images, 26 backgrounds, 0 corrupt: 100%|██████████| 770/770 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/HRW_Relevant-8/valid/labels.cache... 220 images, 10 backgrounds, 0 corrupt: 100%|██████████| 220/220 [00:00<?, ?it/s]


Plotting labels to /content/custom_training_results/train0/train02/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000769, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to /content/custom_training_results/train0/train02
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        220        433      0.902      0.838      0.901      0.655



50 epochs completed in 0.287 hours.
Optimizer stripped from /content/custom_training_results/train0/train02/weights/last.pt, 6.3MB
Optimizer stripped from /content/custom_training_results/train0/train02/weights/best.pt, 6.3MB

Validating /content/custom_training_results/train0/train02/weights/best.pt...
Ultralytics 8.3.44 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,007,403 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:07<00:00,  1.03s/it]


                   all        220        433      0.705      0.832      0.749      0.544
        Links-abbiegen         42         45      0.639      0.864      0.773      0.651
       Rechts-abbiegen         34         36       0.47      0.833      0.486      0.389
            Ampel-grün         47         93      0.835      0.763      0.809      0.488
             Ampel-rot         29         49      0.769      0.543      0.617      0.314
            Ampel-gelb         23         46      0.854      0.913      0.923       0.58
             Geradeaus         44         49      0.832      0.837      0.882      0.675
  Geradeaus-oder-links         24         27      0.484      0.926      0.652      0.524
 Geradeaus-oder-rechts         37         40      0.642      0.895      0.641      0.521
         Zebrastreifen         30         48      0.821      0.917      0.954      0.754
Speed: 0.4ms preprocess, 12.0ms inference, 0.0ms loss, 4.8ms postprocess per image
Results saved to /content/c

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a4a55492e90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.0470

In [12]:
model = YOLO('/content/custom_training_results/train0/train02/weights/best.pt')  # Pfad zum trainierten Modell

# Validierung durchführen
model.val(
    data='/content/HRW_Relevant-8/data.yaml',  # Pfad zur data.yaml-Datei
    save_json=True,                     # Ergebnisse im COCO-Format speichern
    project='/content/custom_training_results/train0/train02',  # Hauptordner für Ergebnisse
    name='Valid_02'           # Name des Validierungsordners
)

Ultralytics 8.3.44 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,007,403 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/HRW_Relevant-8/valid/labels.cache... 220 images, 10 backgrounds, 0 corrupt: 100%|██████████| 220/220 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:04<00:00,  3.34it/s]


                   all        220        433      0.912      0.823      0.899      0.668
        Links-abbiegen         42         45      0.974      0.889      0.963      0.821
       Rechts-abbiegen         34         36      0.945      0.833      0.948      0.764
            Ampel-grün         47         93       0.88      0.712      0.845      0.516
             Ampel-rot         29         49      0.867      0.551      0.628      0.317
            Ampel-gelb         23         46      0.929      0.854      0.916      0.576
             Geradeaus         44         49      0.901      0.816      0.922       0.71
  Geradeaus-oder-links         24         27      0.958      0.926      0.963      0.763
 Geradeaus-oder-rechts         37         40       0.92        0.9      0.948      0.783
         Zebrastreifen         30         48      0.831      0.924      0.953      0.762
Speed: 0.3ms preprocess, 4.3ms inference, 0.0ms loss, 3.2ms postprocess per image
Saving /content/custom_train

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a4a5d341390>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.0470

In [13]:
model = YOLO('/content/custom_training_results/train0/train02/weights/best.pt')  # Pfad zum trainierten Modell
results = model.predict(
    source='/content/HRW_Relevant-8/test/images',  # Pfad zu Testbildern
    conf=0.5,                              # Mindestkonfidenz
    save=True,                              # Ergebnisse speichern
    project='/content/custom_training_results/train0/train02',  # Hauptordner
    name='test_02',                    # Name des Testordners
)


image 1/110 /content/HRW_Relevant-8/test/images/00000.jpg: 640x640 2 Ampel-gelbs, 13.4ms
image 2/110 /content/HRW_Relevant-8/test/images/00001.jpg: 640x640 1 Zebrastreifen, 9.2ms
image 3/110 /content/HRW_Relevant-8/test/images/00002.jpg: 640x640 1 Rechts-abbiegen, 3 Ampel-rots, 9.2ms
image 4/110 /content/HRW_Relevant-8/test/images/00003.jpg: 640x640 1 Geradeaus-oder-links, 14.2ms
image 5/110 /content/HRW_Relevant-8/test/images/00004.jpg: 640x640 1 Rechts-abbiegen, 14.1ms
image 6/110 /content/HRW_Relevant-8/test/images/00005.jpg: 640x640 1 Links-abbiegen, 12.4ms
image 7/110 /content/HRW_Relevant-8/test/images/00006.jpg: 640x640 2 Ampel-gelbs, 8.9ms
image 8/110 /content/HRW_Relevant-8/test/images/00007.jpg: 640x640 1 Geradeaus-oder-rechts, 9.0ms
image 9/110 /content/HRW_Relevant-8/test/images/00008.jpg: 640x640 2 Geradeaus-oder-linkss, 11.1ms
image 10/110 /content/HRW_Relevant-8/test/images/00009.jpg: 640x640 2 Ampel-grüns, 1 Geradeaus, 1 Geradeaus-oder-rechts, 9.0ms
image 11/110 /conte

In [14]:
import shutil
from google.colab import files

# Verzeichnis mit den exportierten Dateien (ersetzen Sie dies durch Ihren Exportpfad)
export_path = '/content/custom_training_results'
zip_path = '/content/exported_model.zip'

# ZIP-Datei erstellen
shutil.make_archive(base_name='/content/exported_model', format='zip', root_dir=export_path)

# ZIP-Datei herunterladen
files.download(zip_path)

print(f"Die ZIP-Datei wurde erfolgreich erstellt und steht unter {zip_path} bereit.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Die ZIP-Datei wurde erfolgreich erstellt und steht unter /content/exported_model.zip bereit.


In [20]:
import shutil

# Ordnerpfad
folder_path = "/content/custom_training_results"

# Ordner löschen
shutil.rmtree(folder_path, ignore_errors=True)

print(f"Der Ordner '{folder_path}' wurde erfolgreich gelöscht.")


Der Ordner '/content/custom_training_results' wurde erfolgreich gelöscht.


In [16]:


from roboflow import Roboflow
rf = Roboflow(api_key="OC1w5MMsOvtMMyW92Jp7")
project = rf.workspace("taleb-alhassan-kiofa").project("hrw_relevant")
version = project.version(9)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to HRW_Relevant-9 in yolov8:: 100%|██████████| 2690/2690 [00:00<00:00, 3557.92it/s]


In [17]:
import os
import random

# Funktion zum Mischen der Dateien in einem Ordner
def shuffle_files(image_dir, label_dir):
    image_files = sorted([f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])
    label_files = sorted([f for f in os.listdir(label_dir) if f.endswith('.txt')])

    # Überprüfen, ob die Anzahl der Bilder und Labels gleich ist
    assert len(image_files) == len(label_files), "Die Anzahl der Bilder und Labels stimmt nicht überein!"

    # Zufällige Reihenfolge erstellen
    indices = list(range(len(image_files)))
    random.shuffle(indices)

    # Dateien umbenennen
    for i, idx in enumerate(indices):
        new_image_name = f"{i:05d}.jpg"  # Benenne die Bilder um
        new_label_name = f"{i:05d}.txt"  # Benenne die Labels um

        os.rename(os.path.join(image_dir, image_files[idx]), os.path.join(image_dir, new_image_name))
        os.rename(os.path.join(label_dir, label_files[idx]), os.path.join(label_dir, new_label_name))

# Ordnerpfade definieren
train_image_dir = "/content/HRW_Relevant-9/train/images"
train_label_dir = "/content/HRW_Relevant-9/train/labels"

valid_image_dir = "/content/HRW_Relevant-9/valid/images"
valid_label_dir = "/content/HRW_Relevant-9/valid/labels"

test_image_dir = "/content/HRW_Relevant-9/test/images"
test_label_dir = "/content/HRW_Relevant-9/test/labels"

# Shuffle für Train, Valid, Test
shuffle_files(train_image_dir, train_label_dir)
shuffle_files(valid_image_dir, valid_label_dir)
shuffle_files(test_image_dir, test_label_dir)

print("Dateien erfolgreich gemischt!")

Dateien erfolgreich gemischt!


In [21]:
from ultralytics import YOLO
import os

# YOLOv8 Modell laden (YOLOv8n für schnelles Training)
model = YOLO('yolov8n.pt')

# Trainieren des Modells mit Augmentierungen
model.train(
    data='/content/HRW_Relevant-9/data.yaml',  # Pfad zur data.yaml-Datei
    epochs=50,                         # Anzahl der Epochen
    batch=16,                          # Batch-Größe
    imgsz=640,                         # Bildgröße
    augment=False,                      # Unaktiviert Augmentierungen
    project='/content/custom_training_results/train1',  # Projektordner
    name='train11',           # Name des spezifischen Trainings
    val=True                            # Validierung nach jeder Epoche
)

Ultralytics 8.3.44 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/HRW_Relevant-9/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/custom_training_results/train1, name=train11, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, 

train: Scanning /content/HRW_Relevant-9/train/labels.cache... 937 images, 26 backgrounds, 0 corrupt: 100%|██████████| 937/937 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/HRW_Relevant-9/valid/labels.cache... 268 images, 10 backgrounds, 0 corrupt: 100%|██████████| 268/268 [00:00<?, ?it/s]


Plotting labels to /content/custom_training_results/train1/train11/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000769, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to /content/custom_training_results/train1/train11
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        268        512      0.746      0.761      0.794      0.614



50 epochs completed in 0.315 hours.
Optimizer stripped from /content/custom_training_results/train1/train11/weights/last.pt, 6.3MB
Optimizer stripped from /content/custom_training_results/train1/train11/weights/best.pt, 6.3MB

Validating /content/custom_training_results/train1/train11/weights/best.pt...
Ultralytics 8.3.44 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,007,403 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:03<00:00,  2.33it/s]


                   all        268        512      0.751      0.764      0.793      0.614
            Ampel-gelb         23         46      0.941       0.87      0.935      0.609
            Ampel-grun         47         93       0.85       0.79      0.827      0.532
             Ampel-rot         29         49      0.826      0.483      0.644      0.371
             Geradeaus         44         49      0.893      0.714      0.882      0.701
  Geradeaus-oder-links         48         51      0.547      0.686      0.729      0.649
 Geradeaus-oder-rechts         54         57      0.608      0.877      0.702      0.597
        Links-abbiegen         59         62      0.699      0.774      0.774      0.681
       Rechts-abbiegen         55         57      0.581      0.802      0.692      0.589
         Zebrastreifen         30         48      0.816      0.875      0.956      0.797
Speed: 0.3ms preprocess, 2.3ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to /content/cu

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a4a5d62d0c0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.0470

In [22]:
model = YOLO('/content/custom_training_results/train1/train11/weights/best.pt')  # Pfad zum trainierten Modell

# Validierung durchführen
model.val(
    data='/content/HRW_Relevant-9/data.yaml',  # Pfad zur data.yaml-Datei
    save_json=True,                     # Ergebnisse im COCO-Format speichern
    project='/content/custom_training_results/train1/train11',  # Hauptordner für Ergebnisse
    name='Valid_11'           # Name des Validierungsordners
)

Ultralytics 8.3.44 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,007,403 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/HRW_Relevant-9/valid/labels.cache... 268 images, 10 backgrounds, 0 corrupt: 100%|██████████| 268/268 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:04<00:00,  4.05it/s]


                   all        268        512      0.748      0.762      0.794      0.613
            Ampel-gelb         23         46       0.94       0.87      0.936      0.609
            Ampel-grun         47         93       0.85       0.79      0.827      0.534
             Ampel-rot         29         49        0.8      0.469      0.644      0.371
             Geradeaus         44         49      0.892      0.714      0.882        0.7
  Geradeaus-oder-links         48         51      0.544      0.686       0.73      0.647
 Geradeaus-oder-rechts         54         57      0.608      0.877      0.702      0.595
        Links-abbiegen         59         62        0.7      0.774      0.774      0.685
       Rechts-abbiegen         55         57      0.581      0.802      0.691      0.588
         Zebrastreifen         30         48      0.815      0.875      0.956      0.791
Speed: 0.3ms preprocess, 4.2ms inference, 0.0ms loss, 2.4ms postprocess per image
Saving /content/custom_train

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a4a1a1077c0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.0470

In [23]:
model = YOLO('/content/custom_training_results/train1/train11/weights/best.pt')  # Pfad zum trainierten Modell
results = model.predict(
    source='/content/HRW_Relevant-9/test/images',  # Pfad zu Testbildern
    conf=0.5,                              # Mindestkonfidenz
    save=True,                              # Ergebnisse speichern
    project='/content/custom_training_results/train1/train11',  # Hauptordner
    name='test_11',                    # Name des Testordners
)


image 1/134 /content/HRW_Relevant-9/test/images/00000.jpg: 640x640 1 Geradeaus-oder-rechts, 8.2ms
image 2/134 /content/HRW_Relevant-9/test/images/00001.jpg: 640x640 1 Rechts-abbiegen, 7.4ms
image 3/134 /content/HRW_Relevant-9/test/images/00002.jpg: 640x640 1 Geradeaus, 7.8ms
image 4/134 /content/HRW_Relevant-9/test/images/00003.jpg: 640x640 1 Ampel-rot, 7.4ms
image 5/134 /content/HRW_Relevant-9/test/images/00004.jpg: 640x640 1 Geradeaus-oder-links, 8.3ms
image 6/134 /content/HRW_Relevant-9/test/images/00005.jpg: 640x640 2 Ampel-gelbs, 7.4ms
image 7/134 /content/HRW_Relevant-9/test/images/00006.jpg: 640x640 1 Links-abbiegen, 7.4ms
image 8/134 /content/HRW_Relevant-9/test/images/00007.jpg: 640x640 1 Rechts-abbiegen, 7.4ms
image 9/134 /content/HRW_Relevant-9/test/images/00008.jpg: 640x640 1 Geradeaus-oder-links, 1 Geradeaus-oder-rechts, 1 Rechts-abbiegen, 8.2ms
image 10/134 /content/HRW_Relevant-9/test/images/00009.jpg: 640x640 (no detections), 7.4ms
image 11/134 /content/HRW_Relevant-9/

In [24]:
model = YOLO('yolov8n.pt')


model.train(
    data='/content/HRW_Relevant-9/data.yaml',  # Pfad zur data.yaml-Datei
    epochs=50,                                 # Anzahl der Epochen
    batch=16,                                  # Batch-Größe
    imgsz=640,                                 # Bildgröße
    augment=True,                              # Aktiviert Augmentierungen
    degrees=5.0,                              # Rotation (±10 Grad)
    translate=0.2,                             # Translation (±20%)
    scale=0.5,                                 # Skalierung (±50%)
    hsv_h=0.015,                               # Farbtonanpassung (±1.5%)
    hsv_s=0.7,                                 # Sättigungsanpassung (±70%)
    hsv_v=0.4,                                 # Helligkeitsanpassung (±40%)
    mosaic=1.0,                                # Mosaic-Augmentierung aktiviert
    mixup=0.2,                                 # MixUp-Augmentierung aktiviert (20%)
    copy_paste=0.1,                            # CopyPaste-Augmentierung aktiviert (10%)
    fliplr=0,                                  # Deaktiviert horizontales Spiegeln
    flipud=0,
    shear =0,
    perspective=0,                            # Keine Perspektivänderung
    project='/content/custom_training_results/train1',  # Projektordner
    name='train12',           # Name des spezifischen Trainings
    val=True
)

Ultralytics 8.3.44 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/HRW_Relevant-9/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/custom_training_results/train1, name=train12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, s

train: Scanning /content/HRW_Relevant-9/train/labels.cache... 937 images, 26 backgrounds, 0 corrupt: 100%|██████████| 937/937 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/HRW_Relevant-9/valid/labels.cache... 268 images, 10 backgrounds, 0 corrupt: 100%|██████████| 268/268 [00:00<?, ?it/s]


Plotting labels to /content/custom_training_results/train1/train12/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000769, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to /content/custom_training_results/train1/train12
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        268        512      0.901      0.849      0.908      0.679



50 epochs completed in 0.341 hours.
Optimizer stripped from /content/custom_training_results/train1/train12/weights/last.pt, 6.3MB
Optimizer stripped from /content/custom_training_results/train1/train12/weights/best.pt, 6.3MB

Validating /content/custom_training_results/train1/train12/weights/best.pt...
Ultralytics 8.3.44 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,007,403 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:04<00:00,  1.86it/s]


                   all        268        512      0.719      0.802      0.737      0.542
            Ampel-gelb         23         46      0.926      0.815      0.918      0.575
            Ampel-grun         47         93      0.818       0.71      0.816      0.479
             Ampel-rot         29         49      0.836      0.519      0.646      0.351
             Geradeaus         44         49      0.875      0.816      0.906      0.671
  Geradeaus-oder-links         48         51      0.499      0.882       0.61      0.529
 Geradeaus-oder-rechts         54         57      0.554      0.849      0.565      0.469
        Links-abbiegen         59         62       0.57      0.871       0.61       0.53
       Rechts-abbiegen         55         57      0.497      0.849      0.597      0.492
         Zebrastreifen         30         48      0.897      0.908      0.969       0.78
Speed: 0.2ms preprocess, 8.4ms inference, 0.0ms loss, 2.8ms postprocess per image
Results saved to /content/cu

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a4b2f2f5c00>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.0470

In [25]:
model = YOLO('/content/custom_training_results/train1/train12/weights/best.pt')  # Pfad zum trainierten Modell

# Validierung durchführen
model.val(
    data='/content/HRW_Relevant-9/data.yaml',  # Pfad zur data.yaml-Datei
    save_json=True,                     # Ergebnisse im COCO-Format speichern
    project='/content/custom_training_results/train1/train12',  # Hauptordner für Ergebnisse
    name='Valid_12'           # Name des Validierungsordners
)

Ultralytics 8.3.44 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,007,403 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/HRW_Relevant-9/valid/labels.cache... 268 images, 10 backgrounds, 0 corrupt: 100%|██████████| 268/268 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:04<00:00,  4.22it/s]


                   all        268        512      0.896      0.852      0.908      0.689
            Ampel-gelb         23         46      0.974      0.821      0.909      0.581
            Ampel-grun         47         93      0.807      0.796      0.817      0.489
             Ampel-rot         29         49      0.739      0.551      0.678      0.351
             Geradeaus         44         49      0.854      0.878      0.914       0.69
  Geradeaus-oder-links         48         51          1        0.9      0.962      0.819
 Geradeaus-oder-rechts         54         57      0.963      0.908      0.975      0.829
        Links-abbiegen         59         62      0.949       0.91      0.966      0.834
       Rechts-abbiegen         55         57      0.939      0.965      0.981      0.809
         Zebrastreifen         30         48      0.835      0.938      0.968      0.801
Speed: 0.3ms preprocess, 4.1ms inference, 0.0ms loss, 2.3ms postprocess per image
Saving /content/custom_train

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a4a38140970>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.0470

In [26]:
model = YOLO('/content/custom_training_results/train1/train12/weights/best.pt')  # Pfad zum trainierten Modell
results = model.predict(
    source='/content/HRW_Relevant-9/test/images',  # Pfad zu Testbildern
    conf=0.5,                              # Mindestkonfidenz
    save=True,                              # Ergebnisse speichern
    project='/content/custom_training_results/train1/train12',  # Hauptordner
    name='test_12',                    # Name des Testordners
)


image 1/134 /content/HRW_Relevant-9/test/images/00000.jpg: 640x640 1 Geradeaus-oder-links, 9.0ms
image 2/134 /content/HRW_Relevant-9/test/images/00001.jpg: 640x640 1 Rechts-abbiegen, 7.4ms
image 3/134 /content/HRW_Relevant-9/test/images/00002.jpg: 640x640 1 Geradeaus, 7.4ms
image 4/134 /content/HRW_Relevant-9/test/images/00003.jpg: 640x640 1 Ampel-rot, 7.4ms
image 5/134 /content/HRW_Relevant-9/test/images/00004.jpg: 640x640 1 Geradeaus-oder-links, 7.4ms
image 6/134 /content/HRW_Relevant-9/test/images/00005.jpg: 640x640 2 Ampel-gelbs, 7.4ms
image 7/134 /content/HRW_Relevant-9/test/images/00006.jpg: 640x640 1 Links-abbiegen, 7.4ms
image 8/134 /content/HRW_Relevant-9/test/images/00007.jpg: 640x640 1 Rechts-abbiegen, 7.4ms
image 9/134 /content/HRW_Relevant-9/test/images/00008.jpg: 640x640 1 Geradeaus-oder-rechts, 1 Links-abbiegen, 7.4ms
image 10/134 /content/HRW_Relevant-9/test/images/00009.jpg: 640x640 (no detections), 7.4ms
image 11/134 /content/HRW_Relevant-9/test/images/00010.jpg: 640

In [27]:
import shutil
from google.colab import files

# Verzeichnis mit den exportierten Dateien (ersetzen Sie dies durch Ihren Exportpfad)
export_path = '/content/custom_training_results'
zip_path = '/content/exported_model1.zip'

# ZIP-Datei erstellen
shutil.make_archive(base_name='/content/exported_model1', format='zip', root_dir=export_path)


print(f"Die ZIP-Datei wurde erfolgreich erstellt und steht unter {zip_path} bereit.")


Die ZIP-Datei wurde erfolgreich erstellt und steht unter /content/exported_model1.zip bereit.
